# KIT-Loe-GE Cell Segmentation and Tracking


Simultaneous cell segmentation and tracking method used for our submission as team KIT-Loe-GE to the [Cell Tracking Challenge](http://celltrackingchallenge.net/) in 2022.

The code is publicly available at https://git.scc.kit.edu/kit-loe-ge/embedtrack .

----

Publication:
K. Löffler and M. Mikut (2022). EmbedTrack -- Simultaneous Cell Segmentation and Tracking Through Learning Offsets and Clustering Bandwidths. arXiv preprint. DOI: [10.48550/arXiv.2204.10713](https://doi.org/10.48550/arXiv.2204.10713)

----


## 1) Setting up the environment

Creating the environment, cloning the code and adding some utilities for downloading the CTC data. Everything (data, code, trained models) will be stored in your personal google drive folder, for accessing it later.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd drive/MyDrive/
!git clone https://git.scc.kit.edu/kit-loe-ge/embedtrack.git


/content/drive/MyDrive
fatal: destination path 'embedtrack' already exists and is not an empty directory.


In [3]:
!conda --version

/bin/bash: conda: command not found


In [4]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:37
🔁 Restarting kernel...


In [5]:
!conda env create -f /content/drive/MyDrive/embedtrack/environment.yml

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [6]:
%%shell
eval "$(conda shell.bash hook)" # copy conda command to shell
conda activate venv_embedtrack
which python
python --version


/usr/local/envs/venv_embedtrack/bin/python
Python 3.7.7


In [7]:
!pip install imagecodecs --no-dependencies
!pip install cffi=="1.15.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.0 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 427 kB 32.6 MB/s 
  Attempting uninstall: cffi
    Found existing installation: cffi 1.14.5
    Uninstalling cffi-1.14.5:
      Successfully uninstalled cffi-1.14.5


In [8]:
# Restarting the runtime
get_ipython().kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

**Check cuda is available - otherwise set in colab under "runtime" -> "change runtime type" the runtime from "None" to "GPU"**



In [9]:
import torch
print(torch.cuda.is_available())

True


**Utilities to facilitate downloading data from the Cell Tracking Challenge**


In [ ]:
import requests
import zipfile
import os

def retrieve_ctc_data(url, save_dir):
  zip_file = os.path.join(save_dir, url.split("/")[-1])
  with requests.get(url, stream=True) as req:
    req.raise_for_status()
    with open(zip_file, "wb") as file: 
      for chunk in req.iter_content(chunk_size=8192):
        file.write(chunk)
  print(f"Unzip data set {os.path.basename(zip_file)}")
  with zipfile.ZipFile(zip_file) as z:
    z.extractall(save_dir)
  
  os.remove(zip_file)
      

## 2.) Training and Inference

### 2.1.) Select a data set to do training / inference on

EmbedTrack was tested and trained on the following 2D datasets, as they all provide an additional Silver Truth (ST) which will be processed together with the Gold Truth annotations (GT) to get fully labelled cell segmentation masks with resonable annotation quality: 
"Fluo-N2DH-SIM+",
  "Fluo-C2DL-MSC",
    "Fluo-N2DH-GOWT1",
    "PhC-C2DL-PSC",
    "BF-C2DL-HSC",
    "Fluo-N2DL-HeLa",
    "BF-C2DL-MuSC",
    "DIC-C2DH-HeLa", and
    "PhC-C2DH-U373".

In [2]:
# possible data sets:

#[    "Fluo-N2DH-SIM+",
#    "Fluo-C2DL-MSC",
#    "Fluo-N2DH-GOWT1",
#    "PhC-C2DL-PSC",
#    "BF-C2DL-HSC",
#    "Fluo-N2DL-HeLa",
#    "BF-C2DL-MuSC",
#    "DIC-C2DH-HeLa",
#    "PhC-C2DH-U373",
#]

data_set = "Fluo-N2DH-SIM+"


### 2.2.) Download the selected data set from the Cell Tracking Challenge

In [1]:
% cd /content/drive/MyDrive/embedtrack/
!ls

/content/drive/MyDrive/embedtrack
ctc_raw_data  embedtrack       gifs	    KIT-Loe-GE	models	   results
data	      environment.yml  __init__.py  LICENSE	README.md


In [14]:
import os
from pathlib import Path

ctc_data_url = "http://data.celltrackingchallenge.net"
ctc_metrics_url = "http://public.celltrackingchallenge.net/software/EvaluationSoftware.zip"

training_data_url = os.path.join(ctc_data_url, "training-datasets/")
challenge_data_url = os.path.join(ctc_data_url, "challenge-datasets/")

current_path = Path.cwd()
data_path = current_path / 'ctc_raw_data'
ctc_metrics_path = os.path.join(current_path, "embedtrack", "ctc_metrics", "CTC_eval")

# Download training data set
if not os.path.exists(data_path / "train" / data_set):
  dp = os.path.join(data_path, "train", data_set)
  print(f"Downloading training data set to {dp} ...")
  data_url = training_data_url + data_set + ".zip"
  retrieve_ctc_data(data_url, os.path.join(data_path, "train"))

# Download challenge data set
if not os.path.exists(data_path / "challenge" / data_set):
  dp = os.path.join(data_path, "challenge", data_set)
  print(f"Downloading challenge data set to {dp} ...")
  data_url = challenge_data_url + data_set + ".zip"
  retrieve_ctc_data(data_url, os.path.join(data_path, "challenge"))

# Download evaluation software
if len(os.listdir(ctc_metrics_path)) <= 1:
  print(f"Downloading  ctc metrics to {ctc_metrics_path} ...")
  retrieve_ctc_data(ctc_metrics_url, ctc_metrics_path)
 
 
!chmod -R 755 $ctc_metrics_path

### 2.3.) Train a model for the selected data set

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("Agg")
from embedtrack.train.run_training_pipeline import (
    DataConfig,
    ModelConfig,
    TrainConfig,
    run_pipeline,
)
import os
from pathlib import Path

# data configs

PROJECT_PATH = "/content/drive/MyDrive/embedtrack/"

RAW_DATA_PATH = os.path.join(PROJECT_PATH, "ctc_raw_data/train")
DATA_PATH_DEST = os.path.join(PROJECT_PATH, "data")
MODEL_PATH = os.path.join(PROJECT_PATH, "models")

USE_SILVER_TRUTH = True
TRAIN_VAL_SEQUNCES = ["01", "02"]
TRAIN_VAL_SPLIT = 0.1

N_EPOCHS = 15
# Adam optimizer; normalize images; OneCycle LR sheduler; N epochs
MODEL_NAME = "adam_norm_onecycle_" + str(N_EPOCHS)

if data_set == "Fluo-N2DH-SIM+":
    use_silver_truth = False
else:
    use_silver_truth = USE_SILVER_TRUTH

data_config = DataConfig(
    RAW_DATA_PATH,
    data_set,
    DATA_PATH_DEST,
    use_silver_truth=use_silver_truth,
    train_val_sequences=TRAIN_VAL_SEQUNCES,
    train_val_split=TRAIN_VAL_SPLIT,
)

# train configs
MODEL_SAVE_DIR = os.path.join(
    MODEL_PATH,
    data_set,
    MODEL_NAME,
)
if data_set != "Fluo-C2DL-MSC":
    CROP_SIZE = 256
    TRAIN_BATCH_SIZE = 16
    VAL_BATCH_SIZE = 16
    DISPLAY_IT = 1000

else:
    CROP_SIZE = 512
    TRAIN_BATCH_SIZE = 8
    VAL_BATCH_SIZE = 8
    DISPLAY_IT = 200

CENTER = "medoid"  
RESUME_TRAINING = False
TRAIN_SIZE = None  # train on full train data set; otherwise still training on full data set but only use a fraction of the data per epoch
VAL_SIZE = None  # train on full train data set; otherwise still training on full data set but only use a fraction of the data per epoch
VIRTUAL_TRAIN_BATCH_MULTIPLIER = 1
VIRTUAL_VAL_BATCH_MULTIPLIER = 1
DISPLAY = False

train_config = TrainConfig(
    MODEL_SAVE_DIR,
    crop_size=CROP_SIZE,
    center=CENTER,
    resume_training=RESUME_TRAINING,
    train_size=TRAIN_SIZE,
    train_batch_size=TRAIN_BATCH_SIZE,
    virtual_train_batch_multiplier=VIRTUAL_TRAIN_BATCH_MULTIPLIER,
    val_size=VAL_SIZE,
    val_batch_size=VAL_BATCH_SIZE,
    virtual_val_batch_multiplier=VIRTUAL_VAL_BATCH_MULTIPLIER,
    n_epochs=N_EPOCHS,
    display=DISPLAY,
    display_it=DISPLAY_IT,
)

# model config
INPUT_CHANNELS = 1
N_SEG_CLASSES = [4, 1]
N_TRACK_CLASSES = 2

model_config = ModelConfig(INPUT_CHANNELS, N_SEG_CLASSES, N_TRACK_CLASSES)

run_pipeline(data_config, train_config, model_config)
plt.close("all")


prepare data of Fluo-N2DH-SIM+


  0%|          | 0/135 [00:00<?, ?it/s]

data stored in /content/drive/MyDrive/embedtrack/data
Dataset properies of the `Fluo-N2DH-SIM+` dataset is saved to /content/drive/MyDrive/embedtrack/data/Fluo-N2DH-SIM+/data_properties.json
/content/drive/MyDrive/embedtrack/data/crops
Created new directory : /content/drive/MyDrive/embedtrack/data/crops/Fluo-N2DH-SIM+/train/02/images/
Created new directory : /content/drive/MyDrive/embedtrack/data/crops/Fluo-N2DH-SIM+/train/02/masks/
Created new directory : /content/drive/MyDrive/embedtrack/data/crops/Fluo-N2DH-SIM+/train/02/center-medoid/
Created new directory : /content/drive/MyDrive/embedtrack/data/Fluo-N2DH-SIM+/train/02/center-medoid/


  0%|          | 0/59 [00:00<?, ?it/s]

Cropping of images, instances and centre_images for data_subset = `train/02` done!
Created new directory : /content/drive/MyDrive/embedtrack/data/crops/Fluo-N2DH-SIM+/train/01/images/
Created new directory : /content/drive/MyDrive/embedtrack/data/crops/Fluo-N2DH-SIM+/train/01/masks/
Created new directory : /content/drive/MyDrive/embedtrack/data/crops/Fluo-N2DH-SIM+/train/01/center-medoid/
Created new directory : /content/drive/MyDrive/embedtrack/data/Fluo-N2DH-SIM+/train/01/center-medoid/


...

100%|██████████| 52/52 [01:31<00:00,  1.76s/it]


===> train loss: 1.080255
===> val loss: 3.2297, val iou: 0.6632
=> saving checkpoint


  0%|          | 0/457 [00:00<?, ?it/s]

Starting epoch 13
learning rate: 4.3349975462426465e-05


 11%|█         | 48/457 [01:09<08:59,  1.32s/it]

### 2.4.) Inference using the just trained model

In [1]:
% cd /content/drive/MyDrive/embedtrack/
!ls

/content/drive/MyDrive/embedtrack
ctc_raw_data  embedtrack       gifs	    LICENSE  README.md
data	      environment.yml  __init__.py  models   results


In [15]:
import os
from datetime import datetime
from pathlib import Path
from time import time
import shutil
from embedtrack.ctc_metrics.eval_ctc import calc_ctc_scores
from embedtrack.infer.infer_ctc_data import inference

PROJECT_PATH = "/content/drive/MyDrive/embedtrack/"

RAW_DATA_PATHS = [os.path.join(PROJECT_PATH, "ctc_raw_data/challenge"),
                  os.path.join(PROJECT_PATH, "ctc_raw_data/train")]
MODEL_PATH = os.path.join(PROJECT_PATH, "models")
RES_PATH = os.path.join(PROJECT_PATH, "results")

# Adam optimizer; normalize images; OneCycle LR sheduler; N epochs
MODEL_NAME = "adam_norm_onecycle_15"
BATCH_SIZE = 32

for raw_data_path in RAW_DATA_PATHS:
      for data_id in ["01", "02"]:
          img_path = os.path.join(raw_data_path, data_set, data_id)

          model_dir = os.path.join(MODEL_PATH, data_set, MODEL_NAME)
          if not os.path.exists(model_dir):
              print(f"no trained model for data set {data_set}")
              continue

          # time stamps
          timestamps_trained_models = [
              datetime.strptime(time_stamp, "%Y-%m-%d---%H-%M-%S")
              for time_stamp in os.listdir(model_dir)
          ]
          timestamps_trained_models.sort()
          last_model = timestamps_trained_models[-1].strftime("%Y-%m-%d---%H-%M-%S")
          model_path = os.path.join(model_dir, last_model, "best_iou_model.pth")
          config_file = os.path.join(model_dir, last_model, "config.json")
          t_start = time()
          #inference(img_path, model_path, config_file, batch_size=BATCH_SIZE)
          t_end = time()

          run_time = t_end - t_start
          print(f"Image sequence: {img_path}")
          print(f"Inference Time {img_path}: {run_time}s")

          res_path = os.path.join(RES_PATH, data_set, MODEL_NAME, last_model, os.path.basename(raw_data_path), data_id+"_RES")
          if not os.path.exists(os.path.dirname(res_path)):
            os.makedirs(res_path)
          shutil.move(img_path+"_RES", res_path)
          if os.path.basename(raw_data_path) == "train":
            metrics = calc_ctc_scores(Path(res_path), Path(img_path+"_GT"))
            print(metrics)



Image sequence: /content/drive/MyDrive/embedtrack/ctc_raw_data/challenge/Fluo-N2DH-SIM+/01
Inference Time /content/drive/MyDrive/embedtrack/ctc_raw_data/challenge/Fluo-N2DH-SIM+/01: 1.1920928955078125e-06s
/content/drive/MyDrive/embedtrack/results/Fluo-N2DH-SIM+/adam_norm_onecycle_15/2022-06-28---14-54-53/challenge/01_RES
Image sequence: /content/drive/MyDrive/embedtrack/ctc_raw_data/challenge/Fluo-N2DH-SIM+/02
Inference Time /content/drive/MyDrive/embedtrack/ctc_raw_data/challenge/Fluo-N2DH-SIM+/02: 4.76837158203125e-07s
/content/drive/MyDrive/embedtrack/results/Fluo-N2DH-SIM+/adam_norm_onecycle_15/2022-06-28---14-54-53/challenge/02_RES
Image sequence: /content/drive/MyDrive/embedtrack/ctc_raw_data/train/Fluo-N2DH-SIM+/01
Inference Time /content/drive/MyDrive/embedtrack/ctc_raw_data/train/Fluo-N2DH-SIM+/01: 7.152557373046875e-07s
/content/drive/MyDrive/embedtrack/results/Fluo-N2DH-SIM+/adam_norm_onecycle_15/2022-06-28---14-54-53/train/01_RES
Image sequence: /content/drive/MyDrive/embe

## 3.) Inference using the models submitted to the CTC

Download the trained models submitted to the CTC and use them for inference.

In [4]:
import os

executables_url = "http://public.celltrackingchallenge.net/participants/KIT-Loe-GE.zip"
executables_path = "/content/drive/MyDrive/embedtrack/"

# Download trained models and executables submitted to the CTC
if not os.path.exists(os.path.join(executables_path, "KIT-Loe-GE")):
  dp = os.path.join(executables_path, "KIT-Loe-GE")
  print(f"Downloading trained models and excetuables of KIT-Loe-GE to {dp} ...")
  retrieve_ctc_data(executables_url, executables_path)

In [5]:
# select data set to do inference on
# possible data sets:
#[    "Fluo-N2DH-SIM+",
#    "Fluo-C2DL-MSC",
#    "Fluo-N2DH-GOWT1",
#    "PhC-C2DL-PSC",
#    "BF-C2DL-HSC",
#    "Fluo-N2DL-HeLa",
#    "BF-C2DL-MuSC",
#    "DIC-C2DH-HeLa",
#    "PhC-C2DH-U373",
#]

data_set = "Fluo-N2DH-SIM+"

In [6]:
% cd /content/drive/MyDrive/embedtrack/
!ls

/content/drive/MyDrive/embedtrack
ctc_raw_data  embedtrack       gifs	    KIT-Loe-GE	models	   results
data	      environment.yml  __init__.py  LICENSE	README.md


In [7]:
import os
from pathlib import Path

ctc_data_url = "http://data.celltrackingchallenge.net"
ctc_metrics_url = "http://public.celltrackingchallenge.net/software/EvaluationSoftware.zip"

training_data_url = os.path.join(ctc_data_url, "training-datasets/")
challenge_data_url = os.path.join(ctc_data_url, "challenge-datasets/")

current_path = Path.cwd()
data_path = current_path / 'ctc_raw_data'

# Download training data set
if not os.path.exists(data_path / "train" / data_set):
  dp = os.path.join(data_path, "train", data_set)
  print(f"Downloading training data set to {dp} ...")
  data_url = training_data_url + data_set + ".zip"
  retrieve_ctc_data(data_url, os.path.join(data_path, "train"))

# Download challenge data set
if not os.path.exists(data_path / "challenge" / data_set):
  dp = os.path.join(data_path, "challenge", data_set)
  print(f"Downloading challenge data set to {dp} ...")
  data_url = challenge_data_url + data_set + ".zip"
  retrieve_ctc_data(data_url, os.path.join(data_path, "challenge"))

In [ ]:
import os

from pathlib import Path
from time import time
import shutil
from embedtrack.ctc_metrics.eval_ctc import calc_ctc_scores
from embedtrack.infer.infer_ctc_data import inference

PROJECT_PATH = "/content/drive/MyDrive/embedtrack/"

RAW_DATA_PATHS = [os.path.join(PROJECT_PATH, "ctc_raw_data/challenge"),
                  os.path.join(PROJECT_PATH, "ctc_raw_data/train")]
MODEL_PATH = os.path.join(PROJECT_PATH, "KIT-Loe-GE", "models")
RES_PATH = os.path.join(PROJECT_PATH, "results")

BATCH_SIZE = 32
for raw_data_path in RAW_DATA_PATHS:
      for data_id in ["01", "02"]:
          img_path = os.path.join(raw_data_path, data_set, data_id)

          model_dir = os.path.join(MODEL_PATH, data_set)
          if not os.path.exists(model_dir):
              print(f"no trained model for data set {data_set}")
              continue
          
          model_path = os.path.join(model_dir, "best_iou_model.pth")
          config_file = os.path.join(model_dir, "config.json")
          t_start = time()
          inference(img_path, model_path, config_file, batch_size=BATCH_SIZE)
          t_end = time()

          run_time = t_end - t_start
          print(f"Image sequence: {img_path}")
          print(f"Inference Time {img_path}: {run_time}s")

          res_path = os.path.join(RES_PATH, data_set, "KIT-Loe-GE", os.path.basename(raw_data_path), data_id+"_RES")
          if not os.path.exists(os.path.dirname(res_path)):
            os.makedirs(res_path)
          shutil.move(img_path+"_RES", res_path)
          if os.path.basename(raw_data_path) == "train":
            metrics = calc_ctc_scores(Path(res_path), Path(img_path+"_GT"))
            print(metrics)



`model_dict` dictionary successfully created with: 
 -- num of classes equal to [4, 1, 2], 
 -- input channels equal to 1, 
 -- name equal to 2d
Creating branched erfnet with [4, 1, 2] classes


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Save tracking mask t064.tif
Save tracking mask t063.tif
Save tracking mask t062.tif
Save tracking mask t061.tif
Save tracking mask t060.tif
Save tracking mask t059.tif
Save tracking mask t058.tif
Save tracking mask t057.tif
Save tracking mask t056.tif
Save tracking mask t055.tif
Save tracking mask t054.tif
Save tracking mask t053.tif
Save tracking mask t052.tif
Save tracking mask t051.tif
Save tracking mask t050.tif
Save tracking mask t049.tif
Save tracking mask t048.tif
Save tracking mask t047.tif
Save tracking mask t046.tif
Save tracking mask t045.tif
Save tracking mask t044.tif
Save tracking mask t043.tif
Save tracking mask t042.tif
Save tracking mask t041.tif
Save tracking mask t040.tif
Save tracking mask t039.tif
Save tracking mask t038.tif
Save tracking mask t037.tif
Save tracking mask t036.tif
Save tracking mask t035.tif
Save tracking mask t034.tif
Save tracking mask t033.tif
Save tracking mask t032.tif
Save tracking mask t031.tif
Save tracking mask t030.tif
Save tracking mask t